In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter

load_dotenv()


True

## Custom Agent 
 Define the function


In [10]:
def abc(num1:int,num2:int)->int:
    "Compute abc between two number"
    return 2*(num1) - 2*(num2)

In [13]:
abc(2,3)

-2

In [16]:
def add_two(num1:int,num2:str):
    return num1 + num2

In [14]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [19]:
abc_json = jsonschema(abc)
abc_json

{'name': 'abc',
 'description': 'Compute abc between two number',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [20]:

model_name = "gpt-3.5-turbo"

In [21]:
client = OpenAI()

response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "compute abc between 2 and 3"},
  ],
)

In [22]:
response.choices[0].message.content

'To compute the average of 2 and 3, you would add the two numbers together and then divide by 2:\n\n(2 + 3) / 2 = 5 / 2 = 2.5\n\nTherefore, the result is 2.5.'

In [23]:
messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass th function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[abc_json],
    function_call="auto",
    temperature=0
)

In [24]:
response

ChatCompletion(id='chatcmpl-AHQvDS9gUaqvlHzT34QCbuEIiO269', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"num1":2,"num2":3}', name='abc'), tool_calls=None))], created=1728718427, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=72, total_tokens=92, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [25]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

FunctionCall(arguments='{"num1":2,"num2":3}', name='abc')
{"num1":2,"num2":3}
<class 'str'>


In [26]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

Function name: abc
Function arguments: {'num1': 2, 'num2': 3}
<class 'dict'>


In [27]:
if func_name == 'abc':
    result = abc(**func_args)
print(result)

-2


# Langchain 

In [32]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [33]:
abc.description

'Compute abc between two numbers'

In [35]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [38]:
tools = [abc]
llm_with_tools = llm.bind_tools(tools)

In [39]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")

In [42]:
response.additional_kwargs

{'tool_calls': [{'id': 'call_q6J6NJpM63Du40duxNh64ZST',
   'function': {'arguments': '{"num1":2,"num2":3}', 'name': 'abc'},
   'type': 'function'}],
 'refusal': None}